In [1]:
import SimpleITK as sitk
import pandas as pd
import os
import numpy as np
import matplotlib.colors as mcolors
import csv

In [2]:
#
# open structure properties file
#
csv_file = 'structure_properties.csv'
df = pd.read_csv(csv_file)
df.set_index('acronym',inplace=True)

In [3]:
# output directory
output_directory = 'landmarks_v05/ccf_annotation'

In [4]:
#
# open annotation file
#
model_directory = "C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST/"
annotation_file = os.path.join( model_directory,  "25um_V3", "CCFTemplate25umLabels.nii.gz")

annotation = sitk.ReadImage( annotation_file )

In [5]:
def create_mask( structure, annotation, df, fg = 1 ) :
    
    slist = df.loc[structure]['substructures']
    slist = slist.split('/')
    slist = [ int(x) for x in slist ]
    
    arr = sitk.GetArrayFromImage( annotation )
    idx = np.isin(arr,slist)
    output = np.zeros(arr.shape,np.uint16)
    output[idx] = fg
    
    output_image = sitk.GetImageFromArray( output )
    output_image.CopyInformation( annotation )    
    return output_image

In [6]:
def overlay( base, image ) :
    
    barr = sitk.GetArrayFromImage( base )
    iarr = sitk.GetArrayFromImage( image )
    
    idx = np.where( iarr > 0 )
    barr[idx] = iarr[idx]
    
    output_image = sitk.GetImageFromArray( barr )
    output_image.CopyInformation( base )    
    return output_image

In [7]:
def set_foreground_value( image, fg ) :
    
    arr = sitk.GetArrayFromImage( image )
    idx = np.where( arr > 0 )
    arr[idx] = fg
    
    output_image = sitk.GetImageFromArray( arr )
    output_image.CopyInformation( image )    
    return output_image    

In [8]:
def change_value( image, old_value, new_value ) :
    
    arr = sitk.GetArrayFromImage( image )
    idx = np.where( arr == old_value )
    arr[idx] = new_value
    
    output_image = sitk.GetImageFromArray( arr )
    output_image.CopyInformation( image )    
    return output_image  

In [9]:
def interface( image1, image2, fg ) :

    mask1 = set_foreground_value( image1, fg )
    mask2 = set_foreground_value( image2, fg )
    
    mask1 = sitk.BinaryDilate( mask1, (1,1,1), sitk.sitkBall, 0, fg )
    mask2 = sitk.BinaryDilate( mask2, (1,1,1), sitk.sitkBall, 0, fg )
    
    arr1 = sitk.GetArrayFromImage( mask1 )
    arr2 = sitk.GetArrayFromImage( mask2 )
    
    idx = np.logical_and( arr1 == fg, arr2  == fg )
    output = np.zeros(arr1.shape,np.uint16)
    output[idx] = fg
 
    output_image = sitk.GetImageFromArray( output )
    output_image.CopyInformation( mask1 )    
    return output_image

In [10]:
def overlap( image1, image2, fg ) :
    
    mask1 = set_foreground_value( image1, fg )
    mask2 = set_foreground_value( image2, fg )
    
    arr1 = sitk.GetArrayFromImage( mask1 )
    arr2 = sitk.GetArrayFromImage( mask2 )
    
    idx = np.logical_and( arr1 == fg, arr2  == fg )
    output = np.zeros(arr1.shape,np.uint16)
    output[idx] = fg
 
    output_image = sitk.GetImageFromArray( output )
    output_image.CopyInformation( mask1 )    
    return output_image    

In [11]:
def connected_components( image, size_threshold = 1000, fg = 1 ) :
    
    components = sitk.ConnectedComponent( image, True )
    arr = sitk.GetArrayFromImage( components )
    values, counts = np.unique( arr, return_counts = True )
    
    # order these by size descending
    ind = np.argsort(counts)
    values = values[np.flip(ind)]
    counts = counts[np.flip(ind)]
    
    output_images = []
    
    for v,c in zip(values,counts) :
        
        # exclude zero label
        if v == 0 :
            continue
        
        # remove components with size below threshold
        if c < 1000 :
            continue
            
        # create an image for the label
        idx = np.where( arr == v )
        output = np.zeros( arr.shape, np.uint16 )
        output[idx] = fg
        
        img = sitk.GetImageFromArray( output )
        img.CopyInformation( image )
        output_images.append( img )
        
    return output_images
    

In [12]:
labels = {}
labels['Clear Label'] = (0, 'black')
labels['mid sagittal'] = ( 1, 'orangered' )
labels['root'] = ( 2, 'firebrick' )
labels['grey'] = (3, 'darkslateblue')
labels['fiber tracts'] = (4, 'olive')
labels['VS'] = (5, 'lime')
labels['fi'] = (6, 'gold')
labels['chpl'] = (7, 'salmon' )
labels['hbc'] = (8, 'springgreen' )
labels['chpl posterior'] = ( 9, 'blueviolet' )
labels['chpl lateral'] = ( 10, 'aqua' )
labels['chpl anterior'] = ( 11, 'orchid' )
labels['och opt'] = (12, 'dodgerblue')
labels['CP cc'] = (13, 'lightseagreen')
labels['CA DG ProS HATA SUB'] = (14, 'crimson')

In [13]:
#
# create label file for ITK-SNAP
#
columns=['index','red','green','blue','transparency','2d_visibility','3d_visibility','label']
snap_df = pd.DataFrame( columns=columns)
snap_df['label'] = list(labels.keys())
snap_df['transparency'] = 1
snap_df['2d_visibility'] = 1
snap_df['3d_visibility'] = 1
snap_df.set_index('label', inplace=True)

for k in labels :

    snap_df.loc[k,'index'] = labels[k][0]
    rgb = mcolors.to_rgb(labels[k][1])
    
    snap_df.loc[k,'red'] = int(rgb[0] * 255)
    snap_df.loc[k,'green'] = int(rgb[1] * 255)
    snap_df.loc[k,'blue'] = int(rgb[2] * 255)
    
snap_df.reset_index(inplace=True)
snap_df = snap_df.reindex(columns=columns)

snap_df.to_csv(os.path.join(output_directory,'landmark_description.txt'), index=False, header=False, sep="\t", quoting=csv.QUOTE_NONNUMERIC)

In [14]:
#
# -- midsagittal mask
#
mid_sagittal = sitk.Image(annotation.GetSize(), sitk.sitkUInt16)
mid_sagittal[:,:,:] = 0
mid_sagittal[227:229,:,:] = labels['mid sagittal'][0]

In [15]:
#
# --- main classes: root/outside, grey matter, fiber tracts and ventricular system
#
root = create_mask('root', annotation, df, labels['root'][0] )
grey = create_mask('grey', annotation, df, labels['grey'][0] )
fiber_tracts = create_mask( 'fiber tracts', annotation, df, labels['fiber tracts'][0] )
VS = create_mask('VS', annotation, df, labels['VS'][0] )

In [16]:
#
# -- clean up stray annotation in root map
#
root[222:235,286:339,75:130] = labels['grey'][0]
root[220:236,286:304,171:204] = labels['grey'][0]

In [17]:
# --- reclass the interface of MB and CB as root/outside
MB = create_mask('MB', annotation, df, labels['grey'][0] )
CB = create_mask('CB', annotation, df, labels['grey'][0] )
MB_CB_interface = interface( MB, CB, fg = labels['root'][0] )

In [18]:
# --- reclass the interface of PAG and CBX as ventricles
PAG = create_mask('PAG', annotation, df, labels['grey'][0] )
CBX = create_mask('CBX', annotation, df, labels['grey'][0] )
PAG_CBX_interface = interface( PAG, CBX, labels['VS'][0] )

In [19]:
# -- reclass the interface of P and CBX as ventricles
P = create_mask('P', annotation, df, labels['grey'][0] )
P_CBX_interface = interface( P, CBX, labels['VS'][0] )

In [20]:
# -- reclass the interface of MY and CBX as root/outside
MY = create_mask('MY', annotation, df, labels['grey'][0] )
MY_CBX_interface = interface( MY, CBX, labels['root'][0] )

In [21]:
# -- reclass the interface of CTXpl and MB as root/outside
CTXpl = create_mask('CTXpl', annotation, df, labels['grey'][0] )
MB_CTXpl_interface = interface( MB, CTXpl, labels['root'][0] )

In [22]:
# -- reclass the interface of TH and CTXpl as root/outside
TH = create_mask('TH', annotation, df, labels['grey'][0] )
TH_CTXpl_interface = interface( TH, CTXpl, labels['root'][0] )

In [23]:
# -- reclass PVpo on the mid-sagittal mask to VS
PVpo = create_mask('PVpo', annotation, df, labels['grey'][0])
PVpo_mid_sagittal_masked = overlap( PVpo, mid_sagittal, labels['VS'][0] )

In [24]:
# -- reclass OLF on the mid-sagittal mask to root/outside
OLF = create_mask('OLF', annotation, df, labels['grey'][0])
OLF_mid_sagittal_masked = overlap( OLF, mid_sagittal, labels['root'][0] )

In [25]:
# -- reclass the bsc as root/outside
bsc = create_mask('bsc', annotation, df, labels['fiber tracts'][0] )
bsc_as_root = set_foreground_value( bsc, labels['root'][0] )

In [26]:
# --make fimbra a distinct landmark
fi = create_mask('fi', annotation, df, labels['fi'][0] )

In [27]:
# -- make hbc a distinct landmark
hbc = create_mask('hbc', annotation, df, labels['hbc'][0] )

In [28]:
# -- make och + opt a distinct landmark
och = create_mask('och', annotation, df, labels['och opt'][0] )
opt = create_mask('opt', annotation, df, labels['och opt'][0] )

In [29]:
# - divide choriod plexeus into 3 distinct landmarks
# -- posterior (largest)
# -- lateral (reunify the component in each hemisphere)
# -- anterior (smallest above threshold)
chpl = create_mask('chpl', annotation, df, labels['chpl'][0] )

components = connected_components( chpl, 1000, labels['chpl'][0] )
chpl_posterior = set_foreground_value( components[0], labels['chpl posterior'][0] )
chpl_lateral = set_foreground_value( overlay( components[1], components[2] ), labels['chpl lateral'][0] )
chpl_anterior = set_foreground_value( components[3], labels['chpl anterior'][0] )

In [30]:
# -- mnake CP + cc a distinct landmark
CP = create_mask('CP', annotation, df, labels['CP cc'][0])
fa = create_mask('fa', annotation, df, labels['CP cc'][0])
ccg = create_mask('ccg', annotation, df, labels['CP cc'][0])
ccb = create_mask('ccb', annotation, df, labels['CP cc'][0])
ccs = create_mask('ccs', annotation, df, labels['CP cc'][0])
fp = create_mask('fp', annotation, df, labels['CP cc'][0])

# -- mask out for any section posterior to 222
CP[:,:222,:] = 0
fa[:,:222,:] = 0
ccg[:,:222,:] = 0
ccb[:,:222,:] = 0
ccs[:,:222,:] = 0
fp[:,:222,:] = 0

In [31]:
# -- mnake CA+DG+SUB distinct landmark
CA = create_mask('CA', annotation, df, labels['CA DG ProS HATA SUB'][0])
DG = create_mask('DG', annotation, df, labels['CA DG ProS HATA SUB'][0])
SUB = create_mask('SUB', annotation, df, labels['CA DG ProS HATA SUB'][0])
ProS = create_mask('ProS', annotation, df, labels['CA DG ProS HATA SUB'][0])
HATA = create_mask('HATA', annotation, df, labels['CA DG ProS HATA SUB'][0])

In [32]:
# -- combined the landmarks into single volume
combined = overlay( root,     grey )
combined = overlay( combined, MB_CB_interface )
combined = overlay( combined, PAG_CBX_interface )
combined = overlay( combined, P_CBX_interface )
combined = overlay( combined, MY_CBX_interface )
combined = overlay( combined, MB_CTXpl_interface )
combined = overlay( combined, TH_CTXpl_interface )
combined = overlay( combined, PVpo_mid_sagittal_masked )
combined = overlay( combined, OLF_mid_sagittal_masked )
combined = overlay( combined, fiber_tracts)
combined = overlay( combined, bsc_as_root )
combined = overlay( combined, hbc )
combined = overlay( combined, fi )
combined = overlay( combined, och )
combined = overlay( combined, opt )
combined = overlay( combined, VS)
combined = overlay( combined, chpl_posterior )
combined = overlay( combined, chpl_lateral )
combined = overlay( combined, chpl_anterior )
combined = overlay( combined, CP )
combined = overlay( combined, fa )
combined = overlay( combined, ccg )
combined = overlay( combined, ccb )
combined = overlay( combined, ccs )
combined = overlay( combined, fp )
combined = overlay( combined, CA )
combined = overlay( combined, DG )
combined = overlay( combined, SUB )
combined = overlay( combined, ProS )
combined = overlay( combined, HATA )

In [33]:
# -- write landmark volume to file
sitk.WriteImage( combined, os.path.join(output_directory,'ccf_landmarks.nii.gz'), True)

In [34]:
#
#  -- create simplified landmarks volume
#
mappings = {}
mappings["fiber tracts"] = 'grey'
mappings['VS'] = 'root'
mappings['chpl lateral'] = 'root'

simplified = combined
simplified.MakeUnique()

for m in mappings :
    
    simplified = change_value( simplified, labels[m][0], labels[mappings[m]][0] )

In [35]:
# -- write simplified landmark volume to file
sitk.WriteImage( simplified, os.path.join(output_directory,'ccf_simplified_landmarks.nii.gz'), True)

In [36]:
#
# -- fill the background as root
#
filled = change_value( simplified, 0, labels['root'][0])
sitk.WriteImage( filled, os.path.join(output_directory,'ccf_simplified_landmarks_filled.nii.gz'), True)

In [37]:
#
# -- get outline
#
mask = sitk.Equal( filled, labels['root'][0] )
mask = set_foreground_value( mask, labels['root'][0] )
outlined = sitk.BinaryContour( mask, False, 0, labels['root'][0] )
sitk.WriteImage( outlined, os.path.join(output_directory,'ccf_outline_with_cuts.nii.gz'), True)